<a href="https://colab.research.google.com/github/aszxcnm22/eee/blob/main/(Drop_%E0%B8%A3%E0%B8%B9%E0%B8%9B%E0%B8%A0%E0%B8%B2%E0%B8%9E).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# การ "ดรอปรูปภาพ" (Drop รูปภาพ) ในบริบทของการเตรียมข้อมูลเพื่อฝึกสอนโมเดล หมายถึง การคัดกรองและตัดทิ้งภาพที่ไม่เหมาะสมหรือไม่มีคุณภาพ


## 🔹 วัตถุประสงค์ของการดรอปรูปภาพ
ลด noise ในข้อมูลที่อาจทำให้โมเดลเรียนรู้ผิดพลาด

ป้องกัน overfitting จากข้อมูลที่ไม่เป็นตัวแทนของปัญหาที่แท้จริง

ทำให้โมเดลสามารถเรียนรู้ได้จากข้อมูลที่ "สะอาด" และมีคุณภาพ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###  ตัวอย่างเกณฑ์ในการดรอปรูปภาพ
ภาพเบลอ / ไม่ชัดเจน

เช่น กล้องสั่น ภาพไม่โฟกัส ทำให้โมเดลมองไม่เห็นลักษณะสำคัญ

In [25]:
import cv2
import numpy as np
import os

def laplacian_variance(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def brenner_gradient(image):
    # คำนวณ Brenner gradient
    shifted = np.roll(image, -2, axis=0)
    diff = (image[:-2, :] - shifted[:-2, :]) ** 2
    return np.sum(diff)

def is_blurry(image_path, lap_threshold=100, brenner_threshold=3000):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return True  # รูปเปิดไม่ติด ถือว่าใช้ไม่ได้

    # ลดขนาดภาพลง (resize) เพื่อความเร็ว (ถ้าต้องการ)
    small_image = cv2.resize(image, (0,0), fx=0.5, fy=0.5)

    lap_var = laplacian_variance(small_image)
    brenner_val = brenner_gradient(small_image)

    # ตรวจสอบค่า threshold ทั้งสอง
    if lap_var < lap_threshold or brenner_val < brenner_threshold:
        return True
    return False

# ตัวอย่างการใช้งาน
image_folder = '/content/drive/MyDrive/Data set  are train'
output_folder = '/content/drive/MyDrive/Data set  are test'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    path = os.path.join(image_folder, filename)
    if not is_blurry(path):
        cv2.imwrite(os.path.join(output_folder, filename), cv2.imread(path))
    else:
        print(f"ดรอปรูปเบลอ: {filename}")


ดรอปรูปเบลอ: image_1_1.png
ดรอปรูปเบลอ:  image_1_4.png
ดรอปรูปเบลอ: image_1_3.png
ดรอปรูปเบลอ: image_1_2.png
ดรอปรูปเบลอ:  image_1_0_1.png
ดรอปรูปเบลอ: image_1_0.png


### ตัวอย่างเกณฑ์ในการดรอปรูปภาพ
ภาพซ้ำซ้อน

ภาพที่เหมือนกันหลายภาพเกินไปอาจทำให้ข้อมูลไม่หลากหลาย

In [5]:
!pip install imagehash Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 4.1 MB/s eta 0:00:00


In [33]:
import os
from PIL import Image
import imagehash
import shutil

# ตั้งค่า
image_folder = '/content/drive/MyDrive/Data set  are train'
output_folder = '/content/drive/MyDrive/Data_set_are_testset'

# ตรวจสอบโฟลเดอร์เป้าหมาย
os.makedirs(output_folder, exist_ok=True)

# เตรียมเก็บ hash
hashes = {}

# ตรวจสอบโฟลเดอร์ภาพ
if not os.path.exists(image_folder):
    print(f" ไม่พบโฟลเดอร์: {image_folder}")
else:
    print(f" เริ่มตรวจสอบโฟลเดอร์: {image_folder}\n")

    for root, _, files in os.walk(image_folder):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(root, filename)
                try:
                    image = Image.open(path)

                    # ข้ามภาพที่ขนาดเล็ก
                    if image.width < 100 or image.height < 100:
                        print(f" ข้ามภาพ resolution ต่ำ: {filename}")
                        image.close()
                        continue

                    # สร้าง hash
                    hash_val = imagehash.phash(image)
                    image.close()

                    if hash_val in hashes:
                        print(f" พบภาพซ้ำ: {filename} → ลบทิ้ง")
                        os.remove(path)
                    else:
                        hashes[hash_val] = path
                        basename = os.path.basename(path)
                        new_path = os.path.join(output_folder, basename)
                        shutil.copy2(path, new_path)
                        print(f" เก็บภาพไม่ซ้ำ: {basename}")

                except Exception as e:
                    print(f" Error ในการประมวลผล {filename}: {e}")

# สรุปผล
print(f"\n ตรวจสอบเสร็จสิ้น")
print(f" จำนวนภาพไม่ซ้ำที่เก็บไว้: {len(hashes)}")
print(f" จำนวนภาพที่ซ้ำถูกลบ: นับจาก log ด้านบน")


 เริ่มตรวจสอบโฟลเดอร์: /content/drive/MyDrive/Data set  are train

 เก็บภาพไม่ซ้ำ:  image_0_3.png
 เก็บภาพไม่ซ้ำ: image_1_1.png
 เก็บภาพไม่ซ้ำ:  image_1_4.png
 เก็บภาพไม่ซ้ำ: image_1_3.png
 เก็บภาพไม่ซ้ำ:  image_0_2.png
 เก็บภาพไม่ซ้ำ: image_1_2.png
 เก็บภาพไม่ซ้ำ:  image_1_0_1.png
 เก็บภาพไม่ซ้ำ:  image_0_4.png
 เก็บภาพไม่ซ้ำ:  image_0_1.png
 เก็บภาพไม่ซ้ำ:  image_0_0.png

 ตรวจสอบเสร็จสิ้น
 จำนวนภาพไม่ซ้ำที่เก็บไว้: 10
 จำนวนภาพที่ซ้ำถูกลบ: นับจาก log ด้านบน


###  ตัวอย่างเกณฑ์ในการดรอปรูปภาพ
ภาพที่มีขนาดเล็กหรือ resolution ต่ำเกินไป

อาจทำให้รายละเอียดในภาพหายไป ส่งผลให้โมเดลเรียนรู้ได้ยาก

In [34]:
import os
from PIL import Image

#  ตั้งค่า
image_folder = '/content/drive/MyDrive/Data_set_are_testset'
min_width = 200   # ความกว้างขั้นต่ำ (px)
min_height = 200  # ความสูงขั้นต่ำ (px)
delete_small_images = False  # เปลี่ยนเป็น True หากต้องการลบ

# เตรียมเก็บรายการภาพขนาดเล็ก
small_images = []

# ตรวจสอบโฟลเดอร์
if not os.path.exists(image_folder):
    print(f" ไม่พบโฟลเดอร์: {image_folder}")
else:
    print(f" ตรวจสอบขนาดภาพใน: {image_folder}\n")

    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(image_folder, filename)
            try:
                with Image.open(path) as img:
                    width, height = img.size
                    if width < min_width or height < min_height:
                        small_images.append((filename, width, height))
                        print(f" {filename}: {width}x{height} (เล็กเกินไป)")

                        if delete_small_images:
                            os.remove(path)
                            print(f"🗑 ลบภาพ: {filename}")

            except Exception as e:
                print(f"❌ Error เปิดภาพ {filename}: {e}")

    # สรุปผล
    if small_images:
        print(f"\n พบภาพขนาดเล็กทั้งหมด: {len(small_images)}")
    else:
        print("\n ไม่พบภาพที่มีขนาดต่ำกว่าที่กำหนด")


 ตรวจสอบขนาดภาพใน: /content/drive/MyDrive/Data_set_are_testset


 ไม่พบภาพที่มีขนาดต่ำกว่าที่กำหนด


In [24]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm

input_folder = "/content/drive/MyDrive/Data set  are test"
output_folder = "/content/drive/MyDrive/Data set  are testset1"
num_augmentations = 1

os.makedirs(output_folder, exist_ok=True)

transform = A.Compose([
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),             # เพิ่ม noise แบบ Gaussian
    A.Rotate(limit=15, p=0.5),                               # หมุนภาพ ±15 องศา
    A.HorizontalFlip(p=0.5),                                 # พลิกภาพซ้าย-ขวา
    A.VerticalFlip(p=0.2),                                   # พลิกภาพบน-ล่าง
    # #A.RandomBrightnessContrast(brightness_limit=0.2,
    #                             contrast_limit=0.2, p=0.3),  # ปรับความสว่าง/คอนทราสต์
    # #A.ShiftScaleRotate(shift_limit=0.05,
    #                    scale_limit=0.1,
    #                    rotate_limit=10, p=0.4),              # ย้าย/ขยาย/หมุน
    # A.ElasticTransform(p=0.2),                               # บิดภาพแบบยืดหยุ่น
    # A.GridDistortion(p=0.2),                                 # ทำให้ภาพเบี้ยวเป็นตาราง
    # A.RandomCrop(width=200, height=200, p=0.3),              # ครอปภาพแบบสุ่ม
    # A.Resize(224, 224),                                      # Resize ให้เท่ากัน (จำเป็นก่อนเข้าโมเดล)
])


image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for img_name in tqdm(image_files, desc="Processing Images"):
    img_path = os.path.join(input_folder, img_name)

    try:
        image = cv2.imread(img_path)
        if image is None:
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(output_folder, img_name), image)  # save original

        for i in range(num_augmentations):
            augmented = transform(image=image_rgb)["image"]
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
            new_name = f"{os.path.splitext(img_name)[0]}_aug{i+1}.jpg"
            new_path = os.path.join(output_folder, new_name)
            cv2.imwrite(new_path, augmented_bgr)
    except Exception as e:
        print(f"Error processing {img_name}: {e}")


<ipython-input-24-2600396470>:14: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
Processing Images: 100%|██████████| 5/5 [00:00<00:00, 11.39it/s]


In [ ]:
import os
import shutil

#  ตั้งค่า
source_folder = "/content/drive/MyDrive/test_set/test_dogs"  # โฟลเดอร์ต้นฉบับ
target_root_folder = "/content/drive/MyDrive/test_set/split_dogs"  # โฟลเดอร์ปลายทาง
images_per_split = 500  #  จำนวนภาพต่อชุด

#  ฟังก์ชัน: ดึง path รูปจากทุก subfolder
def get_all_image_paths(root_folder, extensions=('.jpg', '.jpeg', '.png')):
    image_paths = []
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith(extensions):
                image_paths.append(os.path.join(dirpath, filename))
    return image_paths

#  ดึงรายการภาพทั้งหมด
all_images = get_all_image_paths(source_folder)

#  แบ่งภาพเป็นชุด
for i in range(0, len(all_images), images_per_split):
    split_images = all_images[i:i+images_per_split]
    split_folder = os.path.join(target_root_folder, f"split_{i // images_per_split}")
    os.makedirs(split_folder, exist_ok=True)

    #  คัดลอกภาพไปยังโฟลเดอร์ย่อย
    for img_path in split_images:
        filename = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(split_folder, filename))

print(f" แบ่งภาพทั้งหมด {len(all_images)} รูป เป็นชุดละ {images_per_split} เสร็จแล้วที่: {target_root_folder}")
